# Day 20
## Part 1


In [2]:
from dataclasses import dataclass
import heapq
import math

@dataclass(eq=True, frozen=True)
class Point:
    x: int
    y: int

    def __add__(self, other):
        return self.__class__(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return self.__class__(self.x - other.x, self.y - other.y)

    def __neg__(self):
        return self.__class__(-self.x, -self.y)

    def __lt__(self, other):
        if self.x < other.x:
            return True
        elif self.x > other.x:
            return False
        else:
            return self.y < other.y

    def __iter__(self):
        yield self.x
        yield self.y

    def __mod__(self, other):
        if isinstance(other, Point):
            return self.__class__(self.x % other.x, self.y % other.y)
        else:
            return self.__class__(self.x % other, self.y % other)
        
    def __mul__(self, multiple):
        return self.__class__(self.x * multiple, self.y * multiple)

N = Point(0, 1)
S = Point(0, -1)
W = Point(-1, 0)
E = Point(1, 0)

DIRECTIONS = {N, E, S, W}

def manhattan(p1, p2):
    return abs(p1.x - p2.x) + abs(p1.y - p2.y)

def parse_data(s):
    grid = {}
    lines = s.strip().splitlines()
    starting_point = None
    end_point = None
    for y, line in zip(range(len(lines) - 1, -1, -1), lines):
        for x, c in enumerate(line):
            grid[Point(x, y)] = c
            if c == "S":
                starting_point = Point(x, y)
            elif c == "E":
                end_point = Point(x, y)
    return grid, starting_point, end_point

def shortest_path(grid, starting_point, end_point):
    q = [(manhattan(starting_point, end_point), 0, starting_point)]
    seen = {}

    while q:
        _, length, p = heapq.heappop(q)
        for d in DIRECTIONS:
            n = p + d
            if n == end_point:
                return length + 1
            if grid[n] == "." and seen.get(n, math.inf) > length + 1:
                seen[n] = length + 1
                heapq.heappush(q, (length + 1 + manhattan(n, end_point), length + 1, n))

test_data = parse_data("""###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############""")

shortest_path(*test_data)

84

In [ ]:
def part_1(data, save_value=100):
    grid, starting_point, end_point = data
    target = shortest_path(grid, starting_point, end_point) - save_value
    q = [(0, starting_point, None, set())]
    cheats = set()
    result = 0
    
    while q:
        length, p, cheat, path_so_far = q.pop()
        if p == end_point and length <= target:
            cheats.add(cheat)
            result += 1
        else:
            for d in DIRECTIONS:
                n = p + d
                if n in grid:
                    l = length + 1
                    h = l + manhattan(n, end_point)
                    if h <= target and n not in path_so_far:
                        if grid[n] in (".", "E") and cheat not in cheats:
                            q.append((l, n, cheat, path_so_far | {n}))
                        elif grid[n] == "#" and cheat is None and n not in cheats:
                            q.append((l, n, n, path_so_far | {n}))

    return len(cheats), result

In [46]:
for t in [2, 4, 6, 8, 10, 12, 20, 36, 38, 40, 64]:
    print(t, part_1(test_data, t))

2 44
4 30
6 16
8 14
10 10
12 8
20 5
36 4
38 3
40 2
64 1


In [51]:
data = parse_data(open("input").read())
part_1(data)

KeyboardInterrupt: 

That is massively inefficient and isn't ending any time soon. How long does it take to find the shortest path?

In [50]:
%%time
shortest_path(*data)

CPU times: user 42.8 ms, sys: 10 ms, total: 52.9 ms
Wall time: 51.9 ms


9468

Not long. Try removing each wall section and finding the shortest path.

In [55]:
import tqdm

def part_1_v2(data, save_value=100):
    grid, starting_point, end_point = data
    target = shortest_path(grid, starting_point, end_point) - save_value
    result = 0
    
    max_x = max(p.x for p in grid)
    max_y = max(p.y for p in grid)
    walls = [
        p for p in grid 
        if grid[p] == "#" 
            and p.x != 0 and p.x != max_x
            and p.y != 0 and p.y != max_y
    ]
    for wall in tqdm.tqdm(walls):
        grid[wall] = "."
        if shortest_path(grid, starting_point, end_point) <= target:
            result += 1
        grid[wall] = "#"

    return result

In [57]:
part_1_v2(data, t)

100%|███████████████████████████████████████████████████████████████████████████████| 9852/9852 [08:03<00:00, 20.36it/s]


1827

That's wrong, the answer is too high. I'm leaving this one for now.

Update: the first version took 18 minutes to get the answer with pypy. 